In [1]:
def get_key():
  with open('20250723_OpenAI_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [2]:
!pip install langchain_openai langchain_community langchain_chroma pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 47.1 MB/s eta 0:00:

In [3]:
import os
import urllib.request
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


In [4]:
os.environ['OPENAI_API_KEY'] = get_key()


In [5]:
# 분석할 PDF 파일을 웹에서 다운로드.
urllib.request.urlretrieve("https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")


('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x7bc9cc9f5610>)

In [6]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model="gpt-4.1", temperature=0.2)  # GPT-4o를 언어 모델로 사용
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")  # 임베딩 모델 사용

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 문서를 300자 단위로 분할
    chunk_overlap=100,  # 문맥 유지를 위해 청크 간 100자 중복
)

# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")  # PDF 문서 로더
documents = loader.load()  # 문서에서 텍스트 추출
chunks = text_splitter.split_documents(documents)  # 문서 분할
vector_store = Chroma.from_documents(chunks, embed_model)  # 추출된 텍스트로 벡터 인덱스 생성


In [7]:
def generate_hypothetical_doc(question: str) -> str:
    """질문에 대한 가상의 이상적인 답변 문서 생성"""
    prompt = f"""주어진 질문에 대해, 마치 실제 문서에서 발췌한 것 같은 이상적인 답변을 작성해주세요.
    단, 구체적인 수치, 날짜, 트렌드와 같은 상세 정보를 포함해야 합니다.

    질문: {question}

    답변:"""

    response = llm.invoke(prompt)
    return response.content


In [8]:
def search_with_hyde(hypothetical_doc: str):
    """가상 문서를 이용해 실제 문서 검색"""
    results = vector_store.similarity_search_with_score(hypothetical_doc, k=4)
    return [
        {
            'content': doc.page_content,
            'score': score
        } for doc, score in results
    ]


In [9]:
def generate_final_answer(question: str, relevant_docs: list) -> str:
    """검색된 문서를 바탕으로 최종 답변 생성"""
    context = "\n\n".join([doc['content'] for doc in relevant_docs])

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
    검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {question}

    답변:"""

    response = llm.invoke(prompt)
    return response.content


In [10]:
def process_query(question: str):
    """전체 HyDE 프로세스"""
    # 1. 가상 문서 생성
    print("1. 가상 문서 생성 중...")
    hypothetical_doc = generate_hypothetical_doc(question)
    print("\n가상 문서:", hypothetical_doc)

    # 2. 가상 문서로 검색
    print("\n2. 실제 문서 검색 중...")
    relevant_docs = search_with_hyde(hypothetical_doc)

    # 3. 최종 답변 생성
    print("\n3. 최종 답변 생성 중...")
    final_answer = generate_final_answer(question, relevant_docs)

    return {
        "hypothetical_doc": hypothetical_doc,
        "retrieved_docs": relevant_docs,
        "final_answer": final_answer
    }


In [11]:
question = "북한에서 강제로 이루어지는 조직 생활은 무엇이 있나요?"
result = process_query(question)

print("\n=== 프로세스 결과 ===")
print("\n[가상 문서]")
print(result["hypothetical_doc"])
print("\n[검색된 문서들]")
for idx, doc in enumerate(result["retrieved_docs"], 1):
    print(f"\n문서 {idx} (유사도 점수: {doc['score']:.4f}):")
    print(doc['content'])
print("\n[최종 답변]")
print(result["final_answer"])


1. 가상 문서 생성 중...

가상 문서: 북한에서 강제로 이루어지는 조직 생활은 주로 주민들의 사상 교육과 사회 통제를 목적으로 하는 다양한 형태의 집단 활동을 포함합니다. 대표적인 예로는 '인민반'과 '청년동맹' 활동이 있습니다.

인민반은 북한의 가장 기초적인 행정 단위로, 보통 20~30가구로 구성됩니다. 인민반 회의는 정기적으로 열리며, 주민들은 이 회의에 참석하여 국가 정책에 대한 교육을 받고, 지역 사회의 문제를 논의합니다. 이러한 회의는 주민들의 사상 통제와 감시의 수단으로 활용됩니다.

청년동맹은 만 14세부터 30세까지의 청년들이 가입해야 하는 조직으로, 공식 명칭은 '사회주의애국청년동맹'입니다. 이 조직은 청년들에게 사회주의 이념을 주입하고, 다양한 집단 활동을 통해 국가에 대한 충성을 강화하는 역할을 합니다. 청년동맹은 정기적인 모임과 행사, 그리고 노동 동원 활동을 통해 청년들을 조직적으로 관리합니다.

이 외에도 북한 주민들은 직장이나 학교에서 '당 세포' 활동에 참여해야 하며, 이는 노동당의 지시를 전달받고, 당의 방침을 실천하는 데 중점을 둡니다. 이러한 조직 생활은 주민들의 일상에 깊숙이 침투해 있으며, 개인의 사생활보다는 집단의 이익을 우선시하는 북한 사회의 특징을 잘 보여줍니다.

2. 실제 문서 검색 중...

3. 최종 답변 생성 중...

=== 프로세스 결과 ===

[가상 문서]
북한에서 강제로 이루어지는 조직 생활은 주로 주민들의 사상 교육과 사회 통제를 목적으로 하는 다양한 형태의 집단 활동을 포함합니다. 대표적인 예로는 '인민반'과 '청년동맹' 활동이 있습니다.

인민반은 북한의 가장 기초적인 행정 단위로, 보통 20~30가구로 구성됩니다. 인민반 회의는 정기적으로 열리며, 주민들은 이 회의에 참석하여 국가 정책에 대한 교육을 받고, 지역 사회의 문제를 논의합니다. 이러한 회의는 주민들의 사상 통제와 감시의 수단으로 활용됩니다.

청년동맹은 만 14세부터 30세까지의 청년들이 가입해야 하는 조직으로, 공식 명칭